In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Fetch S&P 500 data for 2023
tickers = ["AAPL", "MSFT", "GOOGL"]  # Add more tickers as needed
data = yf.download(tickers, start="2023-01-01", end="2023-12-31")['Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Scale data for LSTM
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)


[*********************100%***********************]  3 of 3 completed


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Create sequences for LSTM
def create_sequences(data, window_size=30):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 30
X, y = create_sequences(scaled_data, window_size)

# Split into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    LSTM(50),
    Dense(1)  # Predict next-day return
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10


c:\Users\hasht\anaconda3\envs\sp500\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.2829
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0336
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0148
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0254
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0137
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0147
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0122
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0125
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0117
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0116


In [5]:
# Predict returns for each stock
predicted_returns = model.predict(X_test)  # Shape: (len(X_test), len(tickers))

# Generate signals (1 for buy, -1 for sell) for all stocks
signals = np.where(predicted_returns > 0, 1, -1)  # Shape: (len(X_test), len(tickers))



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [7]:
# Start simulation
initial_cash = 100000
portfolio = {"cash": initial_cash, "holdings": {ticker: 0 for ticker in tickers}}
transaction_cost = 0.001  # 0.1% transaction cost

for i in range(len(signals) - 1):  # Iterate over days
    for j, ticker in enumerate(tickers):  # Iterate over stocks
        signal = signals[i][j]  # Extract signal for the specific stock
        price = data.iloc[i + window_size][ticker]  # Get stock price
        
        if signal == 1:  # Buy
            shares = portfolio["cash"] // (price * (1 + transaction_cost))
            portfolio["holdings"][ticker] += shares
            portfolio["cash"] -= shares * price * (1 + transaction_cost)
        elif signal == -1:  # Sell
            shares = portfolio["holdings"][ticker]
            portfolio["cash"] += shares * price * (1 - transaction_cost)
            portfolio["holdings"][ticker] = 0


# Final portfolio value
final_value = portfolio["cash"] + sum(
    portfolio["holdings"][ticker] * data.iloc[-1][ticker] for ticker in tickers
)
print(f"Final portfolio value: ${final_value:.2f}")


IndexError: index 1 is out of bounds for axis 0 with size 1